# Import The required liberaries

In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Upload And Clean Dataset

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

# Split the Dataset into features and labels

In [7]:
concrete_data_columns = concrete_data.columns

#features
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength

#label
target = concrete_data['Strength']

# Normalize

In [8]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


# Basic Regression Model

In [9]:
# define regression model
def regression_model(input_cols):
    
    # create model
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(input_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [10]:
def model(predictors,target,epochs,random_state):
    
    # Split the data into train and test
    predictors_train, predictors_test, target_train, target_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=random_state)
    
    # Train the model
    model = regression_model(predictors_train.shape[1])
    
    # fit the model on the training data
    hist = model.fit(predictors_train, target_train, epochs=epochs, verbose=0)
    
    # find prediction on the test data
    mse_loss = model.evaluate(predictors_test, target_test, verbose=0)
    
    # find target prediction using the model
    target_pred = model.predict(predictors_test)
    
    # calculate mean square error between model prediction and result
    mean_square_error = mean_squared_error(target_test, target_pred)
    
    return mean_square_error,mse_loss,hist

# Train and Test Model

In [11]:
def train_test_model(total_mean_squared_errors, epochs, predictors_norm,target):
    mean_squared_errors_list = []
    history_list = []
    MSE_list = []
    for i in range(0, total_mean_squared_errors):
        mean_square_error,MSE,hist = model(predictors_norm,target,epochs,random_state=i)
        mean_squared_errors_list.append(mean_square_error)
        history_list.append(hist)
        MSE_list.append(MSE)
    return mean_squared_errors_list,history_list,MSE_list

In [12]:
total_mean_squared_errors = 1
epochs = 50
mean_squared_errors_list,history_list,MSE_list =  train_test_model(total_mean_squared_errors, epochs, predictors_norm,target)
for i in range(len(MSE_list)):
    print("MSE "+str(i+1)+": "+str(MSE_list[i]))
mean_squared_errors_list = np.array(mean_squared_errors_list)
mean = np.mean(mean_squared_errors_list)
standard_deviation = np.std(mean_squared_errors_list)   
print(history_list)
print('\n')
print("Mean And Standard Deviation of 1 epochs "+ "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 117.6489011894152


Mean And Standard Deviation of 1 epochs 

Mean: 117.64890469749048
Standard Deviation: 0.0


In [13]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors_list,history_list,MSE_list = train_test_model(total_mean_squared_errors, epochs, predictors_norm,target)
for i in range(len(MSE_list)):
    print("MSE "+str(i+1)+": "+str(MSE_list[i]))
mean_squared_errors_list = np.array(mean_squared_errors_list)
mean = np.mean(mean_squared_errors_list)
standard_deviation = np.std(mean_squared_errors_list)    
print('\n')
print("Mean And Standard Deviation of 50 epochs "+ "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 117.09620036893678
MSE 2: 143.27910053537116
MSE 3: 127.85315652495449
MSE 4: 83.52382798179454
MSE 5: 105.64926532634253
MSE 6: 129.60772803840513
MSE 7: 159.72082381263905
MSE 8: 128.68565230385
MSE 9: 145.78928891277621
MSE 10: 124.37531095338099
MSE 11: 123.4735704440515
MSE 12: 103.12984967771857
MSE 13: 136.96846420633753
MSE 14: 142.91491151087493
MSE 15: 131.29782548922938
MSE 16: 120.94585554499456
MSE 17: 124.50658285887882
MSE 18: 125.90338431052791
MSE 19: 110.83856791277148
MSE 20: 109.3664049562127
MSE 21: 128.7158894955533
MSE 22: 125.48447835483984
MSE 23: 120.96868387858073
MSE 24: 131.48777946299333
MSE 25: 90.36815063081512
MSE 26: 124.16264866702379
MSE 27: 149.0746850566185
MSE 28: 153.8818632946817
MSE 29: 132.80704513105374
MSE 30: 132.05140345304915
MSE 31: 149.4513650542324
MSE 32: 125.48679851791233
MSE 33: 124.21352714859552
MSE 34: 133.59148839067873
MSE 35: 137.25797338393127
MSE 36: 135.22959648058253
MSE 37: 142.34092855607807
MSE 38: 113.645693263